# Open-Ollama-RAG-ChatApp Notebook

## Setup

In [1]:
# %pip install langchain
# %pip install unstructured
# %pip install chromadb
# %pip install langchain_community
# %pip install gradio

In [ ]:
## initial database?
# -> Set to True if you run the notebook for the first time or if you changed the md files
initial_db = False

In [2]:
DATA_PATH = "data/"
OLLAMA_MODEL = "llama2:chat"
OLLAMA_URL = "http://localhost:11434"
CHROMA_PATH = "chroma/"

## langchain split config
# md headers
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("####", "Header 4"),
]

# chunk sizes
chunk_size = 500
chunk_overlap = 100

## Create chunks from md files

In [3]:
from langchain.document_loaders import TextLoader
import os

documents = []

for file in os.listdir(DATA_PATH):
    loader = TextLoader(DATA_PATH + file)
    documents.append(loader.load()[0])

In [4]:
documents[15].metadata
len(documents[15].page_content)

852

In [5]:
# for doc in documents:
#     print(doc.metadata)

In [6]:
from langchain.text_splitter import MarkdownHeaderTextSplitter


text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
chunks_array= []


for doc in documents:
    chunks_array.append(text_splitter.split_text(doc.page_content))
    # append source metadata to each chunk
    for chunk in chunks_array[-1]:
        # combine metadate
        chunk.metadata = doc.metadata



In [7]:
len(chunks_array)

69

In [8]:
# Char-level splits
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len, add_start_index=True
)


In [9]:
chunks_array_txt_base = []
counter = 0
for document in chunks_array:
    for chunk in document:
        splits = text_splitter.split_documents([chunk])
        chunks_array_txt_base.append(splits)
        counter += 1
        

In [10]:
print(counter)
len(chunks_array_txt_base)

345


345

In [11]:
all_document_chunks = [chunk for document in chunks_array_txt_base for chunk in document]

In [12]:
print(len(all_document_chunks))

471


In [13]:
all_document_chunks[1].page_content

'## Thema  \n### Das Thema oder die Frage in einem Satz:\n> Was braucht man für Hardware und was kostet es wenn man sein eigenes LLAMA zuhause haben möchte.'

## Connect to ollama backend

In [14]:
# start ollama
# import subprocess

# subprocess.Popen("ollama serve", shell=True)

In [15]:
# command = f"ollama pull {OLLAMA_MODEL}"

# process = subprocess.Popen(command, shell=True)

In [16]:
# TEST OLLAMA CONNECTION ##
from langchain_community.llms import Ollama

ollama = Ollama(base_url=OLLAMA_URL, model=OLLAMA_MODEL)

print(ollama("what ist Tamagoyaki?"))

/Users/tree/Documents/Dev/RAG/RAG/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Tamagoyaki is a popular Japanese dish that consists of rolled omelette filled with sweet bean paste or other ingredients. The word "tamago" means "egg," and "yaki" means "grilled" or "cooked." Tamagoyaki is often served as a snack or dessert in Japan, and it is known for its smooth, creamy texture and sweet flavor.

The dish typically consists of a thin layer of beaten eggs that are rolled into a cylindrical shape and filled with sweet bean paste, such as azuki bean paste or anko. The eggs are then rolled up again and cooked in a special pan called a tamagoyaki pan, which has a non-stick surface and is designed to create the characteristic spiral pattern of the dish.

Tamagoyaki can be served on its own or with other Japanese sweets, such as mochi or manju. It is also sometimes used as a filling for other desserts, such as ice cream or cake. Overall, tamagoyaki is a popular and delicious treat in Japan that is enjoyed by people of all ages.


## Create vector db

In [17]:
all_document_chunks[0]

Document(page_content='---\ntags:\n- research\n- master-thesis\naliases:\n- Rechenleistung für Lokale LLMs\nstatus:\ntype: research-note\nsubject: ml\ntopic: "[[020_Topic_Master-Thesis]]"\n---', metadata={'source': 'data/Rechenleistung für Lokale LLMs.md', 'start_index': 0})

In [18]:
# create chroma db or load db from disk
if initial_db:
    from langchain.embeddings import OllamaEmbeddings
    from langchain.vectorstores import Chroma

    chroma_db = Chroma.from_documents(all_document_chunks, OllamaEmbeddings(base_url=OLLAMA_URL, model=OLLAMA_MODEL), persist_directory=CHROMA_PATH)

In [19]:
## load chroma db from disk
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma

chroma_db = Chroma(persist_directory=CHROMA_PATH, embedding_function=OllamaEmbeddings(base_url=OLLAMA_URL, model=OLLAMA_MODEL))

In [20]:
# test similarity search
query = "How to make Japanese Egg Rolls?"

result_docs = chroma_db.similarity_search(query)



In [22]:
for doc in result_docs:
    print(doc)

page_content='8. Serve the Panang Curry with a bowl of steamed rice and garnish it with additional Thai basil leaves and roasted peanuts.' metadata={'source': 'data/Thai Panag Curry from Reddit 🍛.md', 'start_index': 817}
page_content='### FX\nRiser/Crashes (Position, Style):\nEar-Candy(Position, Description):' metadata={'source': 'data/Track-Analyse - Top Down Ansatz.md', 'start_index': 0}
page_content='3. Add the water and coconut milk to the pan and bring it to a boil.\n4. Add the green beans to the pan and let it simmer for 10 minutes.\n5. Meanwhile, cut the tofu into cubes and fry them in oil until golden on both sides.\n6. Add the bell peppers, fried tofu, and Thai basil leaves to the pan and simmer for an additional 5 minutes.\n7. Season the curry with salt, sugar, and soy sauce according to your taste. Stir in the Thai basil leaves.' metadata={'source': 'data/Thai Panag Curry from Reddit 🍛.md', 'start_index': 376}
page_content='x---\ntags:\n- music-production\naliases:\nimg_link

## Gradio frontend

In [23]:
from langchain_community.llms import Ollama



def chat_ollama(message, history):
    # initiate ollama
    ollama = Ollama(base_url=OLLAMA_URL, model=OLLAMA_MODEL)

    # search for similar documents in chroma db
    result_chunks = chroma_db.similarity_search(message)
    
    chroma_knowledge = ""
    for id, chunk in enumerate(result_chunks):
        source_id = id + 1
        chroma_knowledge += "[" + str(source_id) +"] \n" + chunk.page_content + "\n"

    sources = ""
    for id, chunk in enumerate(result_chunks):
        source_id = id + 1
        sources += "[" + str(source_id) + "] \n" + chunk.metadata["source"] + "\n"

    prompt = "Answer the following question using the provided knowledge and the chat history:\n\n###KNOWLEDGE: " + chroma_knowledge + "\n###CHAT-HISTORY: " + str(history) + "\n\n###QUESTION: " + message
    result = ollama(prompt) + "\n\n\nReferences:\n" + sources 

    # print(prompt)
    
    return result

In [24]:
chat_ollama("How to make Japanese Eggrolls", "Hello world")

"Great! Based on the provided knowledge and chat history, here's how to make Japanese Egg Rolls:\n\nIngredients:\n\n* 2 pieces garlic\n* 1 small piece ginger\n* Drizzle of olive oil\n* Handful of shimeji mushrooms\n* 1 cup extra firm tofu\n* 2 tbsp peanut butter\n* 2 cups vegetable stock (https://www.youtube.com/watch?v=AgvFmD6sz9I&t=0s)\n* 1 portion ramen noodles (https://www.youtube.com/watch?v=vPvLw5kDpm8&t=0s)\n* 1 tbsp cane sugar\n* 1 tsp rice vinegar\n* 2 tbsp soy sauce\n* 1 tsp sesame oil\n* 2 tbsp roasted peanuts\n* 1 stick green onion\n* Few sprigs cilantro\n\nInstructions:\n\n1. Heat the olive oil in a pan over medium heat. Add the garlic and ginger and sauté until fragrant.\n2. Add the mushrooms and tofu and stir-fry until the tofu is golden brown.\n3. Add the peanut butter and vegetable stock and stir-fry until the mixture thickens.\n4. Add the ramen noodles and stir-fry until they are cooked through.\n5. Season with soy sauce, rice vinegar, sesame oil, cane sugar, and cila

In [28]:
import gradio as gr
gradio_interface = gr.ChatInterface(
        chat_ollama,
        chatbot=gr.Chatbot(),
        textbox=gr.Textbox(placeholder="Example: How to make Japanese Egg Rolls?", container=False, scale=7),
        title="The Ollama test chatbot",
        description=f"Ask the {OLLAMA_MODEL} chatbot a question!",
        theme='gradio/base', # themes at https://huggingface.co/spaces/gradio/theme-gallery
        retry_btn=None,
        undo_btn="Delete Previous",
        clear_btn="Clear",

)



## Run app

In [29]:
gradio_interface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [27]:
### END OF FILE ###